In [1]:
import random
import networkx as nx
import tsplib95
from mst import MST
import matplotlib.pyplot as plt

In [3]:
def plot_graph(graph : nx.Graph, name : str):
    pos = nx.shell_layout(graph)
    nx.draw(graph, pos, with_labels=False)
    weights = {edge : str(graph.get_edge_data(edge[0], edge[1])["weight"]) for edge in graph.edges}
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=weights)
    plt.savefig(name)
    plt.close()

In [7]:
file =  open('datasets/gr17.tsp')
graph = tsplib95.read(file).get_graph()

mst = MST()
mst_graph = mst.get_mst_k(graph, k = 20)
plot_graph(mst_graph, 'mst.png')

odd_nodes = MST.get_odd_degree_nodes(mst_graph)
subgraph = nx.subgraph(graph, odd_nodes)
plot_graph(subgraph, name='subgraph.png')

new_graph = nx.Graph()
for e in graph.edges:
    new_graph.add_edge(e[0], e[1], weight=-graph.get_edge_data(e[0], e[1])["weight"])

set_matching = nx.max_weight_matching(new_graph, maxcardinality=True)
print(set_matching)

matching_graph = nx.Graph()
for m in set_matching:
    matching_graph.add_edge(m[0], m[1], weight=graph.get_edge_data(m[0], m[1])["weight"])

plot_graph(matching_graph, name="min_matching.png")

{(2, 14), (4, 10), (16, 13), (6, 0), (3, 12), (7, 5), (1, 9), (8, 11)}


In [13]:
def plot_graph(graph : nx.Graph, name : str):
    pos = nx.shell_layout(graph)
    nx.draw(graph, pos, with_labels=False)
    weights = {edge : str(graph.get_edge_data(edge[0], edge[1])["weight"]) for edge in graph.edges}
    nx.draw_networkx_edge_labels(graph, pos, edge_labels=weights)
    plt.savefig(name)
    plt.close()

# Load the graph from a TSPLIB file
file = open('datasets/gr17.tsp')
graph = tsplib95.read(file).get_graph()

# Step 1: Compute the Minimum Spanning Tree (MST) using Kruskal's Algorithm
mst = MST()
mst_graph = mst.get_mst_k(graph, k=20)  # Assuming an MST implementation using Kruskal's algorithm
plot_graph(mst_graph, 'mst.png')

# Step 2: Identify vertices with odd degree and create a subgraph
odd_nodes = MST.get_odd_degree_nodes(mst_graph)
subgraph = nx.subgraph(graph, odd_nodes)
plot_graph(subgraph, name='subgraph.png')

# Step 3: Find the minimum cost perfect matching using Blossom Algorithm
# Using the Blossom algorithm via nx.max_weight_matching
new_graph = nx.Graph()
for e in subgraph.edges:
    new_graph.add_edge(e[0], e[1], weight=-subgraph.get_edge_data(e[0], e[1])['weight'])

set_matching = nx.max_weight_matching(new_graph, maxcardinality=True)
print("Minimum Cost Perfect Matching Pairs:", set_matching)

# Add the matching edges to create a multigraph
matching_graph = nx.Graph()
for m in set_matching:
    matching_graph.add_edge(m[0], m[1], weight=graph.get_edge_data(m[0], m[1])['weight'])

plot_graph(matching_graph, name='matching_graph.png')

# Step 4: Combine the MST and matching graph to form an Eulerian graph
eulerian_graph = nx.MultiGraph(mst_graph)
eulerian_graph.add_edges_from(matching_graph.edges(data=True))
# plot_graph(eulerian_graph, name='eulerian_graph.png')

# Step 5: Compute the Eulerian circuit and shortcut to form the TSP tour
eulerian_circuit = list(nx.eulerian_circuit(eulerian_graph))
tsp_tour = []
visited = set()
total_cost = 0

for u, v in eulerian_circuit:
    if u not in visited:
        tsp_tour.append(u)
        visited.add(u)
    total_cost += graph[u][v]['weight'] if graph.has_edge(u, v) else 0
tsp_tour.append(tsp_tour[0])  # Complete the tour by returning to the start

# Validate using the triangle inequality property
for i in range(len(tsp_tour) - 1):
    a, b, c = tsp_tour[i - 1], tsp_tour[i], tsp_tour[(i + 1) % len(tsp_tour)]
    if graph.has_edge(a, c) and graph[a][c]['weight'] <= graph[a][b]['weight'] + graph[b][c]['weight']:
        continue  # Triangle inequality holds

# Output the TSP tour and its cost
print("Final TSP Tour:", tsp_tour)
print("Final Cost of the TSP Tour:", total_cost)


Minimum Cost Perfect Matching Pairs: {(7, 16), (5, 10), (0, 15), (3, 12), (4, 1)}
Final TSP Tour: [2, 10, 9, 1, 4, 5, 7, 6, 0, 15, 11, 8, 12, 3, 16, 13, 14, 2]
Final Cost of the TSP Tour: 2266
